# Linked Art - Visualise - John Ruskin - Story Map


In [5]:
try:
    import os
except:
    !pip install os
    import os
    
try:
    import json
except:
    !pip install json
    import json

In [6]:
storymap = {
    "storymap": {
    "zoomify": False,
    "attribution": "Tanya Gray",
    "call_to_action": False,
    "call_to_action_text": "",
    "map_as_image": False,
    "map_subdomains": "",
    "map_type": "osm:standard",
    "slides": [
            {
            "date": "",
            "location": {
            },
            "media": {
                "caption": "",
                "credit": "Wikipedia",
                "url": "https://en.wikipedia.org/wiki/John_Ruskin"
                    },
            "text": {
                "headline": "John Ruskin's Travels in Europe",
                "text": ""
                    },
            "type": "overview"
            }]
    }
  
    }

In [7]:
storymap_file ="data/ruskin/storyvis/storymap.json"
storyvisdir="data/ruskin/storyvis/json"

file_list=os.listdir(storyvisdir)

sources = {}
sources["https://www.nga.gov/collection/"] = {"name":"National Gallery of Art","colour": "#E8A798"}
sources["https://clevelandart.org/art/"] = {"name":"Cleveland Museum of Art", "colour":"#EDD59E"}
sources["https://www.philamuseum.org/collection/object/"] = {"name":"Philadelphia Museum of Art","colour":"#6B5876"}
sources["https://www.tate.org.uk/art/artworks/"] = {"name":"Tate Museum","colour":"#A09998"}
sources["https://www.harvardartmuseums.org/collections/object/"] = {"name":"Harvard Art Museum","colour":"#00758F"}
sources["ashmolean"] = {"name":"Ashmolean Museum","colour":"#3f83e8"}
sources["rijk"] = {"name":"Rijksmuseum","colour":"#94b7c3"}

for file in file_list:
    with open( storyvisdir + "/" + file) as json_file:
        artwork = json.load(json_file)
        if "produced_by" not in artwork:
            continue
        if "begin_of_the_begin" not in artwork["produced_by"]["timespan"]:
            continue
        if artwork["produced_by"]["timespan"]["begin_of_the_begin"] in  (1819,""):
            continue
    
        id = artwork["id"]
        credit = ""
        text = ""
        imageurl = ""
        bgcolour = ""
        homepage = ""
        
        for source in list(sources.keys()):
            if source in id:
                credit = sources[source]["name"]
                bgcolour = sources[source]["colour"]
            
        if "referred_to_by" in artwork and len(artwork["referred_to_by"]) > 0 and "content" in artwork["referred_to_by"][0]:  
            text = artwork["referred_to_by"][0]["content"]
      
    
        if artwork["subject_of"][0]["classified_as"][1]["id"] == "http://vocab.getty.edu/aat/300266277":
            homepage = artwork["subject_of"][0]["id"]
            text = text + "<br/><br/><a target='_new' href='" + homepage+ "'>Artwork homepage</a>"
    
        if "representation" in artwork:
            if artwork["representation"][0]["id"] != "":
                imageurl = artwork["representation"][0]["id"]
                if "Harvard" in credit:
                    imageurl = imageurl + "?width=500&height=800"
 
        
        if "begin_of_the_begin" in artwork["produced_by"]["timespan"]:
            begin = artwork["produced_by"]["timespan"]["begin_of_the_begin"]
        try:
            begin = int(begin)
        except:
            begin = ""
    
        if "end_of_the_end" in artwork["produced_by"]["timespan"]:
            end = artwork["produced_by"]["timespan"]["end_of_the_end"]
        try:
            end = int(end)
        except:
            end = ""
            
        if abs(int(begin)-int(end)) > 10:
            continue
            
        coords = artwork["shows"]["represents"][0]["approximated_by"][0]["defined_by"]
        coords = coords.replace("POINT(", "")
        coords = coords.replace(")","")
        coords = coords.split()
        
            
        storymap["storymap"]["slides"].append(
            {
            "background": {
                "color": bgcolour,
                "opacity": 100
                    },
            "date": str(begin),
            
            "location": {
                "lat": float(coords[0]),
                
                "lon": float(coords[1]),
                "zoom": 4
                },
            "media": {
                "caption": artwork["produced_by"]["timespan"]["_label"],
                "credit": credit,
                "url": imageurl 
                },
            "text": {
                "headline": artwork["_label"],
                "text": text
                }
            })
        
        storymap["storymap"]["slides"].sort(key=lambda x: x["date"])
          
text_file = open(storymap_file, 'wt')  
n = text_file.write(json.dumps(storymap,indent=2))
text_file.close()
        
        
        

## Story Map Data Visualisation - John Ruskin's Travels in Europe

### Visualisation Screenshots

<img src="docs/media/img/storymap.png"/>

<img src="docs/media/img/storymap2.png"/>

### Make Story Map Visualisation

In [17]:
%%HTML
<div id="mapdiv" style="width: 1000px; height: 800px;background-color:silver"></div>

<link rel="stylesheet" href="https://cdn.knightlab.com/libs/storymapjs/latest/css/storymap.css">
<script type="text/javascript" src="https://cdn.knightlab.com/libs/storymapjs/latest/js/storymap-min.js"></script>

<script>
// storymap_data can be an URL or a Javascript object
var storymap_data = './data/ruskin/storyvis/storymap.json';

// certain settings must be passed within a separate options object
var storymap_options = {};

var storymap = new KLStoryMap.StoryMap('mapdiv', storymap_data, storymap_options);

</script>

In [20]:
vis="https://uploads.knightlab.com/storymapjs/5f6b08e616fe02c0b5d7f0682cca1a69/john-ruskin-in-europe/index.html"

display(IFrame(vis, width='100%', height=800))